In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

In [2]:
import nest_asyncio

In [3]:
nest_asyncio.apply()

In [5]:
client_one = tf.data.Dataset.from_tensor_slices((
    tf.random.uniform(shape=(10, 5)),
    tf.zeros(shape=(10,))
)).batch(10)

In [6]:
client_two = tf.data.Dataset.from_tensor_slices((
    tf.random.uniform(shape=(10, 5)),
    tf.ones(shape=(10,))
)).batch(10)

In [7]:
client_three = tf.data.Dataset.from_tensor_slices((
    tf.random.uniform(shape=(10, 5)),
    tf.ones(shape=(10,))
)).batch(10)

In [8]:
data = [client_one, client_two, client_three]

In [9]:
def model_fn():
    return tff.learning.from_keras_model(
        tf.keras.Sequential([
            tf.keras.layers.Input(shape=(5,)),
            tf.keras.layers.Dense(1, trainable=False),
            tf.keras.layers.Lambda(lambda x: tf.ones_like(x))
        ]),
        input_spec=client_one.element_spec,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.Accuracy(name='acc'), tf.keras.metrics.AUC(curve='pr', name='auc')]
    )

In [10]:
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0)
)

In [11]:
state = iterative_process.initialize()

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


In [12]:
evaluation = tff.learning.build_federated_evaluation(model_fn)

In [13]:
results = evaluation(state.model, data)

In [14]:
results

OrderedDict([('acc', 0.6666667), ('auc', 0.6666667), ('loss', 5.111079)])